In [34]:
import numpy as np
import pandas as pd
import networkx as nx
import traceback
import os
import matplotlib.pyplot as plt
import math
from ant_defs import Ant
import time
import tsplib95
from ant_defs import observer
from ant_defs import makestatsfile

In [35]:
def findbest(pop: list):
    best = Ant(0)
    best.setTourLength(100000000.0)
    for ant in pop:
        if ant.isTourViable():
            if ant < best:
                best = ant
    return best

==============================================================================

ANT SYSTEM

======================================================================================

In [36]:

def ASNextNodeSelection(graph: nx.DiGraph,viable_paths: list,visited: list,args: dict):
    psum = 0
    pvals = []
    for node in viable_paths:
        tmp = math.pow(graph[visited[-1]][node]["heuristic"],args['heuristic_preference']) + math.pow(graph[visited[-1]][node]["pheromone"],args['pheromone_preference'])
        pvals.append(tmp)
    s = np.sum(pvals)
    probs = [x/s for x in pvals]
    #print(probs)
    next_node = np.random.choice(a=viable_paths,p=probs,replace=False)
    #next_node = viable_paths[np.argmax(probs)]
    #print(next_node)
    #return viable_paths[np.argmax(pvals)]
    return next_node

def ASGlobalUpdate(graph: nx.DiGraph,population: list, args: dict) -> nx.DiGraph:
    for ant in population:
        #proper solutions only
        if ant.isTourViable(): 
            tour = ant.getTour()
            startnodes = tour[:-1]
            endnodes = tour[1:]
            for s,n in zip(startnodes,endnodes):
                graph[s][n]["delta"] = graph[s][n]["delta"] + (args['Qval'] / ant.getTourLength())
    evap_coef = 1-args['evaporation']
    for u, v, weight in graph.edges(data="pheromone"):
        graph[u][v]["pheromone"] = evap_coef*graph[u][v]["pheromone"] + graph[u][v]["delta"]
    nx.set_edge_attributes(graph,0.0,"delta")
    return graph

In [37]:
def AntSystemTSP(problem: nx.DiGraph, pop_count: int, max_cycles: int,stats_file, args: dict) -> Ant:
    start = time.time()
    best_solution_cycle = -1
    stagnation = 0
    consecutive_same_best = 0
    argslocal = args
    problem = problem
    nx.set_edge_attributes(problem,args['startpheromone'],"pheromone")
    population = []
    globalbest = Ant(1)
    globalbest.setTourLength(100000000.0)
    argslocal['globalbest'] = globalbest
    localbest = Ant(1)
    localbest.setTourLength(100000000.0)
    for i in range(0,max_cycles):
        population = []
        for k in range(0,pop_count):
            population.append(Ant(1))
        for j in range(0,len(problem)-1):
            for ant in population:
                visited = ant.getTour()
                # i forgot wtf is going on here
                # ok so we assume the last visited node is our starting node
                # and NX Graph gives us a list of all its neighbours
                # and we check if we were already there
                viable_paths = [x for x in [n for n in problem[visited[-1]]] if x not in visited]
                if len(viable_paths) == 0:
                    if j < len(problem)-1:
                        print("ant invalidated")
                        ant.invalidateTour()
                    pass
                else:
                    next_node = ASNextNodeSelection(problem,viable_paths,visited,argslocal)
                    ant.move(next_node,problem[visited[-1]][next_node]["weight"])
                #ants have moved 1 node ahead
            #most ants now have a solution ready
        for ant in population:
            visited = ant.getTour()
            ant.move(visited[0],problem[visited[-1]][visited[0]]["weight"])
        #post cycle pheromone updates and solution assesment
        #review population
        poptours = [x.getTourLength() for x in population if x.isTourViable()]
        localbest = findbest(population)
        #print(localbest.getTourLength())
        #print(consecutive_same_best)
        if localbest < globalbest:
            globalbest = localbest
            argslocal['globalbest'] = globalbest
            best_solution_cycle = i
            consecutive_same_best = 0
        else:
            consecutive_same_best = consecutive_same_best + 1
        if consecutive_same_best > 10:
            stagnation = i
            break
        problem = ASGlobalUpdate(problem,population,argslocal)
    end = time.time()
    extime = end - start
    observer(stats_file,'AS', pop_count, stagnation, best_solution_cycle,globalbest, extime, argslocal)
    return problem, globalbest

=====================================================================================

ANT COLONY ACO

=====================================================================================

In [38]:
def ACONextNodeSelection(graph: nx.DiGraph,viable_paths: list,visited: list,args: dict):
    pvals = []
    if args['selectionTypeConst'] > np.random.uniform():
        for node in viable_paths:
            tmp = graph[visited[-1]][node]["heuristic"] + math.pow(graph[visited[-1]][node]["pheromone"],args['heuristic_preference'])
            pvals.append(tmp)
        return viable_paths[np.argmax(pvals)]
    else:
        return ASNextNodeSelection(graph,viable_paths,visited,args)

def ACOGlobalPheromoneUpdate(graph: nx.DiGraph,population: list, args: dict):
    evap_coef = 1-args['evaporation']
    best = args['globalbest']
    tour = best.getTour()
    startnodes = tour[:-1]
    endnodes = tour[1:]
    for s,n in zip(startnodes,endnodes):
        graph[s][n]["delta"] += math.pow(best.getTourLength(),-1)
    for u, v, weight in graph.edges(data="pheromone"):
        graph[u][v]["pheromone"] = (evap_coef*graph[u][v]["pheromone"]) + (args['evaporation']*graph[u][v]["delta"])
    nx.set_edge_attributes(graph,0.0,"delta")
    return graph

def ACOLocalUpdateTau0(graph: nx.DiGraph, lastnode, newnode, taboo: list, args: dict):
    evap_coef = 1-args['evaporation']
    one = evap_coef*graph[lastnode][newnode]['pheromone']
    two = args['evaporation']*args['startpheromone']
    graph[lastnode][newnode]["pheromone"] = one + two
    return graph

def ACOLocalUpdateQlearning(graph: nx.DiGraph, lastnode, newnode, taboo: list, args: dict):
    evap_coef = 1-args['evaporation']
    nextnodes = [x for x in [n for n in graph[newnode]] if x not in taboo]
    if len(nextnodes) == 0:
        return ACOLocalUpdateTau0(graph,lastnode,newnode,taboo,args)
    else:
        nextphero = []
        for node in nextnodes:
            nextphero.append(graph[newnode][node]['pheromone'])
        delta = args['ACS_qlearn_gamma']*np.max(nextphero)
        one = evap_coef*graph[lastnode][newnode]['pheromone']
        two = args['evaporation']*delta
        graph[lastnode][newnode]["pheromone"] = one + two
        return graph

In [39]:
def ACO_TSP(problem: nx.DiGraph, pop_count: int, max_cycles: int,stats_file, args: dict) -> Ant:
    start = time.time()
    best_solution_cycle = -1
    stagnation = 0
    consecutive_same_best = 0
    argslocal = args
    problem = problem
    nx.set_edge_attributes(problem,args['startpheromone'],"pheromone")
    population = []
    globalbest = Ant(0)
    globalbest.setTourLength(100000000.0)
    argslocal['globalbest'] = globalbest
    localbest = Ant(0)
    localbest.setTourLength(100000000.0)
    for i in range(0,max_cycles):
        population = []
        for k in range(0,pop_count):
            population.append(Ant(1))
        for j in range(0,len(problem)-1):
            for ant in population:
                visited = ant.getTour()
                viable_paths = [x for x in [n for n in problem[visited[-1]]] if x not in visited]
                if len(viable_paths) == 0:
                    if j < len(problem)-1:
                        ant.invalidateTour()
                    pass
                else:
                    next_node = ACONextNodeSelection(problem,viable_paths,visited,argslocal)
                    ACOLocalUpdateQlearning(problem,visited[-1],next_node,visited,argslocal)
                    ant.move(next_node,problem[visited[-1]][next_node]["weight"])
                #ants have moved 1 node ahead
            #most ants now have a solution ready
        for ant in population:
            visited = ant.getTour()
            ant.move(visited[0],problem[visited[-1]][visited[0]]["weight"])
        #post cycle pheromone updates and solution assesment
        #review population
        localbest = findbest(population)
        if localbest < globalbest:
            globalbest = localbest
            argslocal['globalbest'] = globalbest
            best_solution_cycle = i
            consecutive_same_best = 0
        else:
            consecutive_same_best = consecutive_same_best + 1
        if consecutive_same_best > 10:
            stagnation = i
            break
        problem = ACOGlobalPheromoneUpdate(problem,population,argslocal)
    end = time.time()
    extime = end - start
    observer(stats_file,'ACO', pop_count, stagnation, best_solution_cycle,globalbest, extime, argslocal)
    return problem, globalbest

=====================================================================================

MMAS

=====================================================================================

In [40]:
def MMAStaumax(graph: nx.DiGraph, bestlength: float, args: dict):
    one = 1/(1-args['evaporation'])
    two = 1/bestlength
    max_pheromone = one * two
    args['max_pheromone'] = max_pheromone
    return args

def MMAStaumin(graph: nx.DiGraph, args: dict):
    n = graph.number_of_nodes()
    p = args['best_pheromone_selection_prob']**(1/n)
    av = n/2
    min_pheromone = (args['max_pheromone']*(1-p))/((av-1)*p)
    args['min_pheromone'] = min_pheromone
    return args

def MMASTrailSmoothing(graph: nx.DiGraph, args: dict):
    for u, v, pheromone in graph.edges(data="pheromone"):
        graph[u][v]["pheromone"] = pheromone + args['smoothing_coef']*(args['max_pheromone']-pheromone)
    return graph

def MMASGlobalUpdate(graph: nx.DiGraph,localbest, args: dict) -> nx.DiGraph:
        #proper solutions only
    tour = localbest.getTour()
    startnodes = tour[:-1]
    endnodes = tour[1:]
    for s,n in zip(startnodes,endnodes):
        graph[s][n]["delta"] += args['Qval'] / localbest.getTourLength()
    evap_coef = 1-args['evaporation']
    for u, v, weight in graph.edges(data="pheromone"):
        new_pheromone = evap_coef*graph[u][v]["pheromone"] + graph[u][v]["delta"]
        if new_pheromone > args['max_pheromone']:
            new_pheromone = args['max_pheromone']
        elif new_pheromone < args['min_pheromone']:
            new_pheromone = args['min_pheromone']
        graph[u][v]["pheromone"] = new_pheromone
    nx.set_edge_attributes(graph,0.0,"delta")
    return graph

In [41]:
def MMAS_TSP(problem: nx.DiGraph, pop_count: int, max_cycles: int,stats_file, args: dict) -> Ant:
    start = time.time()
    max_stagnations = 2
    current_stagnations = 0
    best_solution_cycle = -1
    stagnation = 0
    consecutive_same_best = 0
    argslocal = args
    problem = problem
    nx.set_edge_attributes(problem,args['startpheromone'],"pheromone")
    population = []
    globalbest = Ant(0)
    globalbest.setTourLength(100000000.0)
    argslocal['globalbest'] = globalbest
    localbest = Ant(0)
    localbest.setTourLength(100000000.0)
    for i in range(0,max_cycles):
        population = []
        for k in range(0,pop_count):
            population.append(Ant(1))
        for j in range(0,len(problem)-1):
            for ant in population:
                visited = ant.getTour()
                # i forgot wtf is going on here
                # ok so we assume the last visited node is our starting node
                # and NX Graph gives us a list of all its neighbours
                # and we check if we were already there
                viable_paths = [x for x in [n for n in problem[visited[-1]]] if x not in visited]
                if len(viable_paths) == 0:
                    if j < len(problem)-1:
                        ant.invalidateTour()
                    pass
                else:
                    next_node = ASNextNodeSelection(problem,viable_paths,visited,argslocal)
                    ant.move(next_node,problem[visited[-1]][next_node]["weight"])
                #ants have moved 1 node ahead
            #most ants now have a solution ready
        for ant in population:
            visited = ant.getTour()
            ant.move(visited[0],problem[visited[-1]][visited[0]]["weight"])
        #post cycle pheromone updates and solution assesment
        #review population
        localbest = findbest(population)
        if localbest < globalbest:
            globalbest = localbest
            argslocal['globalbest'] = globalbest
            best_solution_cycle = i
            consecutive_same_best = 0
        else:
            consecutive_same_best = consecutive_same_best + 1
        argslocal = MMAStaumax(problem,globalbest.getTourLength(),argslocal)
        argslocal = MMAStaumin(problem,argslocal)
        problem = MMASGlobalUpdate(problem,localbest,argslocal)
        poptours = [x.getTourLength() for x in population if x.isTourViable()]
        if consecutive_same_best > 10:
            stagnation = i
            problem = MMASTrailSmoothing(problem,argslocal)
            consecutive_same_best = 0
            current_stagnations = current_stagnations + 1
        if current_stagnations > max_stagnations:
            break
        # TODO: Find way to detect convergence and stagnation to insert trail smoothing here
    end = time.time()
    extime = end - start
    observer(stats_file,'MMAS', pop_count, stagnation, best_solution_cycle,globalbest, extime, argslocal)
    return problem, globalbest

=====================================================================================

ASRANK

=====================================================================================

In [42]:
def ASRANKGlobalUpdate(graph,population,args):
    good_solutions = [x for x in population if x.isTourViable()]
    good_solutions.sort(reverse=True)
    ranks = args['elite_multiplier']-1
    used_solutions = good_solutions[-ranks:]

    for rank, ant in enumerate(used_solutions):
        r_rank = rank + 1
        tour = ant.getTour()
        startnodes = tour[:-1]
        endnodes = tour[1:]
        for s,n in zip(startnodes,endnodes):
            graph[s][n]["delta"] += (args['Qval'] / ant.getTourLength())*r_rank
    best = args['globalbest']   
    besttour = best.getTour()
    startnodes = besttour[:-1]
    endnodes = besttour[1:]
    for s,n in zip(startnodes,endnodes):
        graph[s][n]["elite_delta"] += (args['Qval'] / best.getTourLength())*args['elite_multiplier']
    evap_coef = 1-args['evaporation']
    for u, v, weight in graph.edges(data="pheromone"):
        graph[u][v]["pheromone"] = evap_coef*graph[u][v]["pheromone"] + graph[u][v]["delta"] + graph[u][v]["elite_delta"]
    nx.set_edge_attributes(graph,0.0,"delta")
    nx.set_edge_attributes(graph,0.0,"elite_delta")
    return graph

In [43]:
def ASRANK_TSP(problem: nx.DiGraph, pop_count: int, max_cycles: int, stats_file, args: dict) -> Ant:
    start = time.time()
    best_solution_cycle = -1
    stagnation = 0
    consecutive_same_best = 0
    argslocal = args
    problem = problem
    nx.set_edge_attributes(problem,args['startpheromone'],"pheromone")
    nx.set_edge_attributes(problem,0.0,"elite_delta")
    population = []
    globalbest = Ant(0)
    globalbest.setTourLength(100000000.0)
    argslocal['globalbest'] = globalbest
    localbest = Ant(0)
    localbest.setTourLength(100000000.0)
    for i in range(0,max_cycles):
        population = []
        for k in range(0,pop_count):
            population.append(Ant(1))
        for j in range(0,len(problem)-1):
            for ant in population:
                visited = ant.getTour()
                # i forgot wtf is going on here
                # ok so we assume the last visited node is our starting node
                # and NX Graph gives us a list of all its neighbours
                # and we check if we were already there
                viable_paths = [x for x in [n for n in problem[visited[-1]]] if x not in visited]
                if len(viable_paths) == 0:
                    if j < len(problem)-1:
                        ant.invalidateTour()
                    pass
                else:
                    next_node = ASNextNodeSelection(problem,viable_paths,visited,argslocal)
                    ant.move(next_node,problem[visited[-1]][next_node]["weight"])
                #ants have moved 1 node ahead
            #most ants now have a solution ready
        #post cycle pheromone updates and solution assesment
        #review population
        for ant in population:
            visited = ant.getTour()
            ant.move(visited[0],problem[visited[-1]][visited[0]]["weight"])
        localbest = findbest(population)
        if localbest < globalbest:
            globalbest = localbest
            argslocal['globalbest'] = globalbest
            best_solution_cycle = i
            consecutive_same_best = 0
        else:
            consecutive_same_best = consecutive_same_best + 1
        if consecutive_same_best > 10:
            stagnation = i
            break
        problem = ASRANKGlobalUpdate(problem,population,argslocal)
    end = time.time()
    extime = end - start
    observer(stats_file,'ASRANK', pop_count, stagnation, best_solution_cycle,globalbest, extime, argslocal)
    return problem, globalbest

In [44]:
stats_file = makestatsfile('ftv70.csv')

In [45]:
p = tsplib95.load('./test_instances/ftv70.atsp')
p.edge_weights  
graph = p.get_graph()
nx.set_edge_attributes(graph,1.0,"pheromone")
nx.set_edge_attributes(graph,'b',"color")
nx.set_edge_attributes(graph,0.0,"heuristic")
nx.set_edge_attributes(graph,0.0,"delta")

for u, v, weight in graph.edges(data="weight"):
    if(weight is not None and weight != 0):
        print(f'source {u}, dest {v}, weight {weight}')
        graph[u][v]["heuristic"] = 1/weight

source 0, dest 0, weight 100000000
source 0, dest 1, weight 26
source 0, dest 2, weight 46
source 0, dest 3, weight 74
source 0, dest 4, weight 82
source 0, dest 5, weight 65
source 0, dest 6, weight 90
source 0, dest 7, weight 104
source 0, dest 8, weight 102
source 0, dest 9, weight 100
source 0, dest 10, weight 147
source 0, dest 11, weight 134
source 0, dest 12, weight 90
source 0, dest 13, weight 75
source 0, dest 14, weight 69
source 0, dest 15, weight 106
source 0, dest 16, weight 117
source 0, dest 17, weight 42
source 0, dest 18, weight 71
source 0, dest 19, weight 96
source 0, dest 20, weight 158
source 0, dest 21, weight 89
source 0, dest 22, weight 76
source 0, dest 23, weight 125
source 0, dest 24, weight 38
source 0, dest 25, weight 40
source 0, dest 26, weight 58
source 0, dest 27, weight 59
source 0, dest 28, weight 13
source 0, dest 29, weight 38
source 0, dest 30, weight 31
source 0, dest 31, weight 22
source 0, dest 32, weight 103
source 0, dest 33, weight 143
source

In [46]:
# from QAPInstances import getBur26a
# burA, burB = getBur26a()
# burA = np.matrix(burA)
# burB = np.matrix(burB)
# vectA = burA.sum(axis=1)
# vectB = burB.sum(axis=1)
# graphweights = np.dot(vectA,vectB.T)
# graph = nx.from_numpy_matrix(graphweights, create_using=nx.DiGraph)
# nx.set_edge_attributes(graph,1.0,"pheromone")
# nx.set_edge_attributes(graph,'b',"color")
# nx.set_edge_attributes(graph,0.0,"heuristic")
# nx.set_edge_attributes(graph,0.0,"delta")

# for u, v, weight in graph.edges(data="weight"):
#     if(weight is not None and weight != 0):
#         print(f'source {u}, dest {v}, weight {weight}')
#         graph[u][v]["heuristic"] = 1/weight

In [47]:
args_AS = {
    'pheromone_preference': 10,
    'heuristic_preference': 10,
    'Qval': 1,
    'evaporation': 0.9,
    'startpheromone': 2.0
}

args_ACO = {
    'pheromone_preference': 10,
    'heuristic_preference': 10,
    'Qval': 1,
    'evaporation': 0.9,
    'startpheromone': 2.0,
    'selectionTypeConst': 0.3,
    'ACS_qlearn_gamma': 0.5
}

args_MMAS = {
    'pheromone_preference': 10,
    'heuristic_preference': 10,
    'Qval': 1,
    'evaporation': 0.9,
    'startpheromone': 2.0,
    'best_pheromone_selection_prob': 0.05,
    'max_pheromone': 4.0,
    'min_pheromone': 0.5,
    'smoothing_coef': 0.5
}

args_ASRANK = {
    'pheromone_preference': 10,
    'heuristic_preference': 5,
    'Qval': 1,
    'evaporation': 0.9,
    'startpheromone': 1.0,
    'elite_multiplier': 35
}

# args_AS = {
#     'pheromone_preference': 1,
#     'heuristic_preference': 1,
#     'Qval': 1,
#     'evaporation': 0.5,
#     'startpheromone': 1.0
# }

# args_ACO = {
#     'pheromone_preference': 1,
#     'heuristic_preference': 1,
#     'Qval': 1,
#     'evaporation': 0.5,
#     'startpheromone': 1.0,
#     'selectionTypeConst': 0.5,
#     'ACS_qlearn_gamma': 0.5
# }

# args_MMAS = {
#     'pheromone_preference': 1,
#     'heuristic_preference': 1,
#     'Qval': 1,
#     'evaporation': 0.5,
#     'startpheromone': 1.0,
#     'best_pheromone_selection_prob': 0.05,
#     'max_pheromone': 2.0,
#     'min_pheromone': 0.5,
#     'smoothing_coef': 0.5
# }

# args_ASRANK = {
#     'pheromone_preference': 1,
#     'heuristic_preference': 1,
#     'Qval': 1,
#     'evaporation': 0.5,
#     'startpheromone': 1.0,
#     'elite_multiplier': 62
# }
# kro124p - 36230
# ft53 - 6905
# ft70 - 1950
# ftv35 - 1473

In [48]:
for d in range(0,5):
    G, solution = AntSystemTSP(graph,70,1000,stats_file,args_AS)
    besttour = solution.getTour()
    print(besttour)
    print(solution.getTourLength())

[1, 2, 70, 3, 4, 5, 0, 28, 25, 27, 6, 7, 9, 11, 10, 58, 61, 59, 8, 57, 56, 52, 46, 62, 49, 48, 51, 50, 67, 47, 39, 40, 68, 54, 55, 53, 44, 43, 42, 41, 37, 63, 38, 36, 34, 35, 69, 60, 33, 32, 26, 22, 64, 17, 66, 14, 13, 12, 15, 16, 18, 21, 23, 19, 20, 24, 29, 30, 31, 65, 45, 1]
2206.0
[1, 2, 5, 0, 28, 64, 17, 66, 14, 12, 13, 27, 6, 7, 9, 11, 10, 58, 61, 59, 8, 57, 56, 70, 3, 4, 25, 15, 16, 18, 21, 22, 23, 60, 33, 35, 34, 69, 41, 37, 63, 40, 68, 46, 62, 49, 48, 51, 50, 67, 47, 39, 38, 36, 43, 42, 32, 26, 19, 20, 24, 29, 30, 31, 65, 44, 45, 53, 52, 54, 55, 1]
2267.0
[1, 2, 5, 25, 27, 6, 7, 9, 11, 10, 58, 61, 59, 8, 57, 56, 70, 3, 4, 0, 28, 64, 17, 66, 14, 13, 12, 15, 16, 18, 21, 22, 23, 60, 33, 35, 34, 69, 41, 37, 38, 36, 63, 40, 68, 46, 62, 49, 47, 39, 48, 51, 50, 67, 54, 55, 52, 53, 44, 65, 43, 42, 32, 26, 19, 20, 24, 29, 30, 31, 45, 1]
2121.0
[1, 2, 5, 0, 28, 25, 27, 6, 7, 9, 11, 10, 58, 61, 59, 8, 57, 56, 70, 3, 4, 31, 30, 29, 24, 26, 22, 17, 64, 66, 14, 13, 12, 15, 16, 18, 21, 19, 20

In [49]:
for d in range(0,5):
    graph, solution = ACO_TSP(graph,70,1000,stats_file,args_ACO)
    besttour = solution.getTour()
    print(besttour)
    print(solution.getTourLength())

[1, 2, 5, 25, 27, 6, 7, 9, 11, 10, 58, 61, 57, 56, 70, 3, 4, 8, 59, 0, 28, 24, 26, 22, 18, 21, 17, 64, 66, 14, 12, 13, 15, 16, 19, 20, 23, 60, 33, 32, 35, 34, 69, 41, 37, 63, 40, 68, 46, 62, 49, 48, 51, 50, 67, 47, 39, 38, 36, 42, 43, 54, 55, 52, 53, 44, 65, 45, 29, 30, 31, 1]
2234.0
[1, 2, 5, 0, 28, 24, 26, 22, 18, 21, 17, 64, 25, 27, 6, 7, 9, 11, 10, 59, 8, 13, 66, 14, 12, 15, 16, 19, 20, 23, 60, 33, 35, 34, 69, 41, 37, 63, 40, 68, 46, 62, 54, 55, 56, 70, 3, 57, 58, 61, 4, 65, 44, 43, 42, 32, 36, 38, 39, 47, 48, 51, 50, 67, 49, 52, 53, 45, 31, 30, 29, 1]
2209.0
[1, 2, 5, 0, 28, 24, 26, 22, 18, 21, 17, 64, 25, 27, 6, 7, 9, 11, 10, 58, 61, 57, 56, 70, 3, 59, 8, 13, 66, 14, 12, 15, 16, 19, 20, 23, 60, 33, 35, 34, 69, 41, 37, 63, 40, 68, 46, 62, 52, 53, 44, 43, 42, 32, 36, 38, 39, 47, 48, 51, 50, 67, 49, 54, 55, 4, 31, 30, 29, 65, 45, 1]
2182.0
[1, 2, 5, 25, 27, 6, 7, 9, 11, 10, 58, 61, 59, 8, 57, 56, 70, 3, 4, 0, 28, 64, 17, 66, 14, 13, 12, 15, 16, 21, 22, 18, 19, 20, 23, 60, 33, 35, 34

In [50]:
for d in range(0,5):
    graph, solution = MMAS_TSP(graph,70,1000,stats_file,args_MMAS)
    besttour = solution.getTour()
    print(besttour)
    print(solution.getTourLength())

[1, 2, 5, 0, 28, 64, 17, 66, 14, 12, 15, 16, 18, 21, 22, 19, 20, 23, 60, 33, 35, 34, 69, 41, 37, 63, 40, 68, 46, 62, 49, 48, 51, 50, 67, 47, 39, 42, 32, 26, 24, 25, 27, 6, 7, 9, 13, 11, 10, 58, 61, 59, 8, 57, 56, 70, 3, 4, 30, 29, 31, 65, 44, 43, 38, 36, 54, 55, 52, 53, 45, 1]
2245.0
[1, 2, 5, 0, 28, 25, 27, 6, 7, 9, 11, 10, 58, 61, 57, 56, 70, 3, 4, 8, 59, 52, 46, 62, 54, 55, 53, 44, 43, 42, 41, 37, 63, 40, 68, 49, 48, 51, 50, 67, 47, 39, 38, 36, 34, 35, 69, 60, 33, 32, 26, 22, 17, 64, 66, 14, 13, 12, 15, 16, 18, 21, 23, 19, 20, 24, 29, 30, 31, 65, 45, 1]
2217.0
[1, 2, 5, 0, 28, 24, 26, 22, 18, 21, 17, 64, 25, 27, 6, 7, 9, 13, 66, 14, 12, 15, 16, 19, 20, 23, 60, 32, 33, 35, 34, 69, 41, 37, 63, 40, 68, 46, 62, 49, 48, 51, 50, 67, 47, 39, 38, 36, 43, 42, 54, 55, 56, 70, 3, 4, 8, 59, 57, 58, 61, 11, 10, 52, 53, 44, 65, 45, 30, 29, 31, 1]
2204.0
[1, 2, 5, 0, 28, 24, 26, 22, 18, 21, 17, 64, 25, 27, 6, 7, 9, 13, 14, 12, 15, 16, 19, 20, 23, 60, 33, 32, 34, 35, 69, 41, 37, 63, 40, 68, 46, 62,

In [51]:
for d in range(0,5):
    graph, solution = ASRANK_TSP(graph,70,1000,stats_file,args_ASRANK)
    besttour = solution.getTour()
    print(besttour)
    print(solution.getTourLength())

[1, 2, 5, 0, 28, 24, 26, 22, 18, 21, 14, 12, 15, 16, 17, 64, 25, 27, 6, 7, 9, 11, 10, 58, 61, 59, 57, 56, 70, 3, 4, 8, 13, 66, 19, 20, 23, 60, 33, 34, 35, 69, 41, 37, 63, 40, 68, 46, 62, 54, 55, 53, 44, 43, 42, 32, 36, 38, 39, 47, 48, 51, 50, 67, 49, 29, 30, 31, 65, 45, 52, 1]
2368.0
[1, 2, 4, 57, 58, 61, 59, 8, 7, 9, 11, 10, 14, 12, 15, 16, 21, 22, 17, 64, 24, 26, 28, 0, 65, 44, 43, 42, 41, 37, 63, 40, 68, 46, 62, 54, 55, 56, 70, 3, 5, 25, 27, 6, 13, 66, 18, 19, 20, 23, 60, 33, 32, 34, 35, 69, 38, 36, 39, 47, 48, 51, 50, 67, 49, 52, 53, 45, 31, 30, 29, 1]
2323.0
[1, 2, 70, 3, 8, 7, 9, 11, 10, 13, 66, 14, 12, 15, 16, 18, 19, 20, 21, 22, 17, 64, 28, 0, 65, 44, 43, 42, 35, 34, 69, 41, 37, 63, 40, 68, 46, 62, 52, 53, 56, 4, 5, 25, 27, 6, 24, 26, 33, 32, 23, 60, 36, 38, 39, 47, 48, 51, 50, 67, 49, 54, 55, 57, 58, 61, 59, 31, 30, 29, 45, 1]
2355.0
[1, 2, 5, 0, 28, 64, 17, 66, 14, 12, 15, 16, 18, 21, 22, 19, 20, 23, 60, 33, 35, 34, 69, 41, 37, 63, 40, 68, 46, 62, 49, 48, 51, 50, 67, 47, 39, 